In [1]:
from copy import deepcopy
from itertools import permutations
from collections import deque

class IntcodeComputer(object):
    
    def __init__(self, program):
        self.mem = deepcopy(program)
        self.i = 0
        self.inputs = deque([])
        self.last_out = None
        self.done = False
        
    def process_instruction(self, instruction):
        instruction = instruction.zfill(2)
        opcode = instruction[-2:]
        param_modes = instruction[:-2]
        
        if opcode == '01':
            self._add(param_modes)
        elif opcode == '02':
            self._mult(param_modes)
        elif opcode == '03':
            self._input(param_modes)
        elif opcode == '04':
            return self._output(param_modes)
        elif opcode == '05':
            self._jump_if_true(param_modes)
        elif opcode == '06':
            self._jump_if_false(param_modes)
        elif opcode == '07':
            self._less_than(param_modes)
        elif opcode == '08':
            self._equals(param_modes)
        elif opcode == '99':
            self.done = True
            return 0
        else:
            raise ValueError(f"Unknown opcode: '{opcode}' from instruction: '{instruction}'")
        
    def _get_param_val(self, param_pos, mode):
        if mode == '0':
            return int(self.mem[int(self.mem[self.i+param_pos])])
        elif mode == '1':
            return int(self.mem[self.i+param_pos])
        else:
            raise ValueError(f'Unknown mode: {mode}')
        
    def _write_param(self, param_pos, mode, value):
        if mode == '0':
            self.mem[int(self.mem[self.i+param_pos])] = str(value)
        elif mode == '1':
            self.mem[self.i+param_pos] = str(value)
        else:
            raise ValueError(f'Unknown mode: {mode}')

        
    def _add(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        self._write_param(3, param_modes[-3], a+b)
        self.i += 4
    
    def _mult(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        self._write_param(3, param_modes[-3], a*b)
        self.i += 4
    
    def _input(self, param_modes):
        param_modes = param_modes.zfill(1)
        self._write_param(1, param_modes[-1], self.inputs.popleft())
        self.i += 2
            
    def _output(self, param_modes):
        param_modes = param_modes.zfill(1)
        out = self._get_param_val(1, param_modes[-1])
        self.i += 2
        self.last_out = out
        return out
        
    def _jump_if_true(self, param_modes):
        param_modes = param_modes.zfill(2)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a != 0:
            self.i = b
        else:
            self.i += 3
            
    def _jump_if_false(self, param_modes):
        param_modes = param_modes.zfill(2)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a == 0:
            self.i = b
        else:
            self.i += 3  
            
    def _less_than(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a < b:
            self._write_param(3, param_modes[-3], 1)
        else:
            self._write_param(3, param_modes[-3], 0)
        self.i += 4
        
    def _equals(self, param_modes):
        param_modes = param_modes.zfill(3)
        a = self._get_param_val(1, param_modes[-1])
        b = self._get_param_val(2, param_modes[-2])
        if a == b:
            self._write_param(3, param_modes[-3], 1)
        else:
            self._write_param(3, param_modes[-3], 0)
        self.i += 4
        
    def run(self):
        while self.i < len(self.mem):
            out = self.process_instruction(self.mem[self.i])
            if out is not None:
                return out

In [2]:
# Part 1

def run_amp(program, _input, phase_setting):
    amp = IntcodeComputer(program)
    amp.inputs.append(phase_setting)
    amp.inputs.append(_input)
    return amp.run()

def run_amp_chain(program, phase_settings):
    _input = 0
    for ps in phase_settings:
        _input = run_amp(program, _input, ps)
        
    return _input
    
def find_highest_output(program):
    best_out = 0
    for permuation in permutations([0, 1, 2, 3, 4], 5):
        _out = run_amp_chain(program, permuation)
        if _out > best_out:
            best_out = _out
            
    return best_out

program = open('input/day7.txt').read().splitlines()[0].split(',')
find_highest_output(program)

338603

In [3]:
# Part 2

def run_amp_chain_2(program, phase_settings):
    _input = 0
    amps = []
    for ps in phase_settings:
        amp = IntcodeComputer(program)
        amp.inputs.append(ps)
        amps.append(amp)
        
    while not amps[4].done:
        for amp in amps:
            amp.inputs.append(_input)
            _input = amp.run()
            
    return amps[4].last_out

def find_highest_output_2(program):
    best_out = 0
    for permuation in permutations([5, 6, 7, 8, 9], 5):
        _out = run_amp_chain_2(program, permuation)
        if _out > best_out:
            best_out = _out
            
    return best_out

find_highest_output_2(program)

63103596